# Progetto IUM - Data Analysis
## Università di Informatica, Unito
Gruppo Federico Raso, Federico Verra

In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [31]:
data_directory = "./Data/"
actors_df = pd.read_csv(data_directory+'actors.csv')
countries_df = pd.read_csv(data_directory+'countries.csv')
crew_df = pd.read_csv(data_directory+'crew.csv')
genres_df = pd.read_csv(data_directory+'genres.csv')
languages_df = pd.read_csv(data_directory+'languages.csv')
movies_df = pd.read_csv(data_directory+'movies.csv')
posters_df = pd.read_csv(data_directory+'posters.csv')
releases_df = pd.read_csv(data_directory+'releases.csv')
reviews_rotten_df = pd.read_csv(data_directory+'rotten_tomatoes_reviews.csv')
studios_df = pd.read_csv(data_directory+'studios.csv')
oscar_df = pd.read_csv(data_directory+'the_oscar_awards.csv')

## Analizzo e pulisco i dati

In [32]:
movies_df

,id,name,date,tagline,description,minute,rating
0,1000001,Barbie,2023.0,She's everything. He's just Ken.,Barbie and Ken are having the time of their li...,114.0,3.86
1,1000002,Parasite,2019.0,Act like you own the place.,"All unemployed, Ki-taek's family takes peculia...",133.0,4.56
2,1000003,Everything Everywhere All at Once,2022.0,The universe is so much bigger than you realize.,An aging Chinese immigrant is swept up in an i...,140.0,4.30
3,1000004,Fight Club,1999.0,Mischief. Mayhem. Soap.,A ticking-time-bomb insomniac and a slippery s...,139.0,4.27
4,1000005,La La Land,2016.0,Here's to the fools who dream.,"Mia, an aspiring actress, serves lattes to mov...",129.0,4.09
...,...,...,...,...,...,...,...
941592,1941593,神笛,NaN,NaN,NaN,NaN,NaN
941593,1941594,蟲極道蜜団子抗争編 壱ノ巻,NaN,NaN,Shinjuku forest at night. In the sap taverns o...,30.0,NaN
941594,1941595,蟲極道蜜団子抗争編 弐ノ巻,NaN,NaN,"The city that never sleeps, where insects gath...",30.0,NaN
941595,1941596,重生,NaN,NaN,"In a world where order has broken down, darkne...",NaN,NaN


In [33]:
movies_df.describe()

,id,date,minute,rating
count,9.415970e+05,849684.000000,760027.000000,90999.000000
mean,1.470799e+06,1998.522300,65.776516,3.244043
std,2.718158e+05,27.534865,154.828161,0.417281
min,1.000001e+06,1874.000000,1.000000,0.880000
25%,1.235400e+06,1986.000000,15.000000,3.020000
50%,1.470799e+06,2010.000000,62.000000,3.300000
75%,1.706198e+06,2019.000000,92.000000,3.510000
max,1.941597e+06,2031.000000,72000.000000,4.690000


In [34]:
movies_df.shape

(941597, 7)

In [35]:
movies_df.columns.values

array(['id', 'name', 'date', 'tagline', 'description', 'minute', 'rating'],
      dtype=object)

In [36]:
movies_df.dtypes

id               int64
name            object
date           float64
tagline         object
description     object
minute         float64
rating         float64
dtype: object

In [45]:
movies_df['date'] = pd.to_datetime(movies_df['date'], errors='coerce', format='%Y').fillna(pd.to_datetime(movies_df['date'],errors='coerce'))

In [48]:
movies_df.dtypes

id                      int64
name                   object
date           datetime64[ns]
tagline                object
description            object
minute                float64
rating                float64
dtype: object

In [53]:
oscar_df

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927-01-01,1928-01-01,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927-01-01,1928-01-01,1,ACTOR,Emil Jannings,The Last Command,True
2,1927-01-01,1928-01-01,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927-01-01,1928-01-01,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927-01-01,1928-01-01,1,ACTRESS,Gloria Swanson,Sadie Thompson,False
...,...,...,...,...,...,...,...
10884,2023-01-01,2024-01-01,96,WRITING (Original Screenplay),Written by Celine Song,Past Lives,False
10885,2023-01-01,2024-01-01,96,JEAN HERSHOLT HUMANITARIAN AWARD,NaN,NaN,True
10886,2023-01-01,2024-01-01,96,HONORARY AWARD,"To Angela Bassett, who has inspired audiences ...",NaN,True
10887,2023-01-01,2024-01-01,96,HONORARY AWARD,"To Mel Brooks, for his comedic brilliance, pro...",NaN,True


In [52]:
oscar_df['year_film'] = pd.to_datetime(oscar_df['year_film'], errors='coerce', format='%Y').fillna(pd.to_datetime(oscar_df['year_film'],errors='coerce'))

oscar_df['year_ceremony'] = pd.to_datetime(oscar_df['year_ceremony'], errors='coerce', format='%Y').fillna(pd.to_datetime(oscar_df['year_ceremony'],errors='coerce'))

In [57]:
releases_df

,id,country,date,type,rating
0,1000001,Andorra,2023-07-21,Theatrical,NaN
1,1000001,Argentina,2023-07-20,Theatrical,ATP
2,1000001,Australia,2023-07-19,Theatrical,PG
3,1000001,Australia,2023-10-01,Digital,PG
4,1000001,Austria,2023-07-20,Theatrical,NaN
...,...,...,...,...,...
1332777,1940967,USA,1909-01-01,Theatrical,NaN
1332778,1940968,Sweden,1908-11-11,Theatrical,NaN
1332779,1940969,France,1902-01-01,Theatrical,NaN
1332780,1940970,France,1902-01-01,Theatrical,NaN


In [56]:
releases_df['date'] = pd.to_datetime(releases_df['date'], errors='coerce', format='%Y').fillna(pd.to_datetime(releases_df['date'],errors='coerce'))

# analizzo i dati